In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from offer_search.ranking.elasticsearch import ElasticsearchRanker

In [3]:
import pandas as pd

In [4]:
data = pd.read_excel('./data/sport/double-sports/data.xls')
data.head()

,Название,Описание,Цена
0,Велосипеды / Велосипеды / Туринги / AUTHOR / В...,Хромолевый грэвел (gravel) Author RONIN 2019 г...,84000
1,Велосипеды / Велосипеды / Горные / BERGAMONT /...,"Универсальный горный велосипед, который отличн...",43450
2,Велосипеды / Велосипеды / Горные / BERGAMONT /...,"Универсальный горный велосипед, который отличн...",43450
3,Велосипеды / Велосипеды / Горные / BERGAMONT /...,В 2018 году компания Bergamont решила объедини...,43450
4,Велосипеды / Велосипеды / Горные / BERGAMONT /...,В 2018 году компания Bergamont решила объедини...,43450


In [5]:
preset = []
for _, row in data.iterrows():
    body = {
        'Item': row['Название'].lower(),
        'Attributes': '' if row['Описание'] != row['Описание'] else row['Описание'],
        'Price': int(row['Цена']),
    }
    preset.append(body)

In [6]:
es = ElasticsearchRanker(preset=preset)

In [9]:
es.rank(
    {
        'Item': 'комбинезон',
        'Attributes': '',
        'Price_from': 0,
        'Price_to': 1000000,
    }
)

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 336,
  'max_score': 5.764652,
  'hits': [{'_index': 'offer_search',
    '_type': 'offer',
    '_id': '3hqAnmgBtrFpTVtEBc14',
    '_score': 5.764652,
    '_source': {'Item': 'одежда / комбинезоны / cool zone / комбинезоны cool zone snowman kn2143/07 волна 2019 /~комбинезоны cool zone snowman kn2143/07 волна 2019',
     'Price': 20900}},
   {'_index': 'offer_search',
    '_type': 'offer',
    '_id': '4BqAnmgBtrFpTVtEBc2H',
    '_score': 5.764652,
    '_source': {'Item': 'одежда / комбинезоны / cool zone / комбинезоны cool zone snowmobile kn2140/20 черный 2019 /~комбинезоны cool zone snowmobile kn2140/20 черный 2019',
     'Price': 21600}},
   {'_index': 'offer_search',
    '_type': 'offer',
    '_id': 'QBqQnmgBtrFpTVtE9djU',
    '_score': 5.764652,
    '_source': {'Item': 'одежда / комбинезоны / cool zone / комбинезоны cool zone snowman kn2143/07 волна 2019 /~комбинез